In [58]:
### IMPORTS ###
import scipy.sparse
import numpy as np
import matplotlib.pyplot as pyplot
import sklearn.metrics as mtc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import TruncatedSVD
from pyemd import emd
import valores



In [59]:
## FUNCIONES ##


class TFM:

    def __init__(self,path):
        self.X_tr = scipy.sparse.load_npz(path+'X_tr.npz')
        self.X_tst = scipy.sparse.load_npz(path+'X_tst.npz')
        self.y_tr = np.load(path+'y_tr.npy')
        self.y_tst = np.load(path+'y_tst.npy')
        # Se binarizan las caracteristicas 
        self.y_tr=MultiLabelBinarizer().fit_transform(self.y_tr)
        self.y_tst=MultiLabelBinarizer().fit_transform(self.y_tst)
        # Escalado 
        scaler = MaxAbsScaler().fit(self.X_tr)
        self.X_tr=scaler.transform(self.X_tr)
        self.X_tst=scaler.transform(self.X_tst)
        self.X_tr_simpli= None
        self.simplificador = False
        self.modelo = None
        self.clf= None
        self.columns= None
        print ('Dataset cargado y preprocesado')

    def reduce_features(self,num_data):
        columns=np.array([])
        for i in range(np.shape(self.X_tr)[1]):
            if self.X_tr[:,i].count_nonzero()> num_data:
                columns=np.append(columns,i)
        np.savetxt('features'+num_data+'.txt',columns,delimiter=',')
        print('Las características han sido reducidas')

    def simplify_dataset(self,features,tr_size):
        self.X_tr_simpli=self.X_tr
        self.y_tr_simpli=self.y_tr
        
        self.columns=np.loadtxt('features'+features+'.txt',delimiter=',')
        self.X_tr_simpli=self.X_tr[:,self.columns]
        """
        self.svd = TruncatedSVD(n_components=int(features), n_iter=5, random_state=42)
        
        self.X_tr_simpli=self.svd.fit_transform(self.X_tr)
        """
        self.X_tr_simpli, __, self.y_tr_simpli, __ = train_test_split(self.X_tr_simpli, self.y_tr_simpli, test_size=(1-tr_size), random_state=42)
        print('Simplificación finalizada')
        print(np.shape(self.X_tr_simpli))

    def use_simplify(self,boolean):
        self.simplificador= boolean
        
    def grid_search_cv(self,classifier,parameters,metodo):
        if (metodo == 0):
            classif = OneVsRestClassifier(classifier)
        elif (metodo==1):
            classif = ClassifierChain(classifier)
        model_tunning = GridSearchCV(classif, param_grid=parameters,cv=4,verbose=50, n_jobs=-1,scoring='accuracy')
        if (self.simplificador):
            model_tunning.fit(self.X_tr_simpli, self.y_tr_simpli)
        else :
            model_tunning.fit(self.X_tr, self.y_tr)
        print (model_tunning.best_score_)
        print (model_tunning.best_params_)
        self.modelo = model_tunning.best_estimator_

    def fitting_classifier(self):
        if (self.simplificador):
            self.clf=self.modelo.fit(self.X_tr,self.y_tr)
        else :
            self.clf=self.modelo

    def metrics(self):
        if (self.simplificador):
            self.y_pred=self.clf.predict(self.X_tst)
        else :
            self.y_pred=self.clf.predict(self.X_tst)
        accuracy=mtc.accuracy_score(self.y_tst,self.y_pred)
        hamming=mtc.hamming_loss(self.y_tst,self.y_pred)
        precision=mtc.precision_score(self.y_tst,self.y_pred,average='micro')
        print("Total accuracy: ", accuracy)
        print("Hamming loss: ", hamming)
        print("Precision: ", precision)
        print(mtc.classification_report(self.y_tst,self.y_pred))
        #print("Accuracy per class:")
        aux=0
        for i in range(np.shape(self.y_pred)[1]):
            #print ("Class " ,i,": " ,mtc.accuracy_score(self.y_tst[:,i],y_pred[:,i]))
            aux=aux+mtc.accuracy_score(self.y_tst[:,i],self.y_pred[:,i])
        print("Accuracy media: ",aux/37)
        distance_matrix=np.ones((37,37))*(1/37)
        np.fill_diagonal(distance_matrix,0)
        print("EMD: ", emd(self.y_tst,self.y_pred,distance_matrix))
        



In [ ]:
path_string=input('Introduce el directorio del dataset')
clasificacion=TFM(path_string)
print('Inicio del programa clasificador')

while True:
    
    ask2=int(input('Que clasificador vas a usar?: '))
    clasificador=valores.clasificadores_dict[ask2]
    metod=int(input('Que metodo vas a usar?: Introduce 0 para One vs Rest, o 1 para Classifier Chain'))
        
    while True:
        ask1=input(('Quieres usar simplificacion para entrenar el algoritmo?'))
        if (ask1.lower()=='si'):
            non_zero_data=input(('Introduce el numero de elementos no nulos por categoria que consideras aceptable'))
            training_size=float(input('Introduce el porcentaje de tamano de datos de entrenamiento que quieres usar para simplificar'))
            print('Realizando simplificacion del dataset...')
            clasificacion.simplify_dataset(non_zero_data,training_size)
            clasificacion.use_simplify(True)
        elif (ask1.lower()=='no'):
            clasificacion.use_simplify(False)
        print("Iniciando tunning de los parámetros:")
        clasificacion.grid_search_cv(clasificador,valores.parametros_dict[(clasificador,metod)],metod)    
        print("Tunning finalizado:")  
        ask5=input('Quieres continuar con el entrenamiento?')
        if (ask5.tolower()=='no'){
            break
        }
        print("Entrenando el algoritmo con los mejores parametros...")
        clasificacion.fitting_classifier()
        print("Entrenamiento finalizado, se procede a predecir las etiquetas")
        print("Mostrando metricas obtenidas")
        clasificacion.metrics()
        ask3=input('Quieres probar a variar la simplificacion?')
        if (ask3.lower()=='si'):
            continue
        elif (ask3.lower()=='no'):
            break
            
    ask4=input('Quieres probar con otro clasificador o finalizamos el programa?')
    if (ask4.lower()=='si'):
        continue
    elif (ask4.lower()=='no'):
        break

print("Fin del programa clasificador")
    



In [ ]:
#path_string='../../Datasets/dataset/'
#mlp=MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(300,200,100,50), random_state=0)
#{'estimator__kernel': 'linear', 'estimator__C': 1, 'estimator__cache_size': 500, 'estimator__gamma': 0.1} 0.3009367681498829


In [60]:
path_string="../../Datasets/dataset/"
metod=0
clasificador=valores.LogisticRegression()
parameters={
    "estimator__C": [1],
    "estimator__solver": ["liblinear"],
    "estimator__tol": [0.01]
}
clasificacion=TFM(path_string)
clasificacion.use_simplify(False)
clasificacion.grid_search_cv(clasificador,parameters,metod)
clasificacion.fitting_classifier()
clasificacion.metrics()

Dataset cargado y preprocesado
Fitting 4 folds for each of 1 candidates, totalling 4 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:  1.9min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  1.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  1.9min finished
0.39788608713238033
{'estimator__C': 1, 'estimator__solver': 'liblinear', 'estimator__tol': 0.01}
Total accuracy:  0.39135317237507017
Hamming loss:  0.02901497792008741
Precision:  0.7681099084096586
              precision    recall  f1-score   support

           0       0.86      0.70      0.77       109
           1       0.78      0.46      0.58       138
           2       0.77      0.64      0.70       224
           3       0.76      0.55      0.64       180
           4       0.82      0.69      0.75       177
           5 

/Users/mariotrapiellomendoza/tfm/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mariotrapiellomendoza/tfm/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [41]:
print (np.ones((37,37))*(1/37))



[[0.02702703 0.02702703 0.02702703 ... 0.02702703 0.02702703 0.02702703]
 [0.02702703 0.02702703 0.02702703 ... 0.02702703 0.02702703 0.02702703]
 [0.02702703 0.02702703 0.02702703 ... 0.02702703 0.02702703 0.02702703]
 ...
 [0.02702703 0.02702703 0.02702703 ... 0.02702703 0.02702703 0.02702703]
 [0.02702703 0.02702703 0.02702703 ... 0.02702703 0.02702703 0.02702703]
 [0.02702703 0.02702703 0.02702703 ... 0.02702703 0.02702703 0.02702703]]


In [62]:
print(np.shape(clasificacion.y_tst))

(1781, 37)
